Clustering Toronto Neighborhoods part 3

add foursquare data to the dataframe then cluster the neighboords
use folium to visalize 

In [5]:

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
from sklearn.cluster import KMeans
import folium 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors
import re

print('Libraries imported.')

Libraries imported.


In [6]:

source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')

#body = soup.find('body')

#print(body.prettify()) 

postcode = soup.find_all("table", attrs={"class":"wikitable"})

In [7]:
table1 = postcode[0]
body = table1.find_all("tr")

head = body[0] # all 0th items are now the header row
body_rows = body[1:]# all other rows are now the body

headings = []
for item in head.find_all("th"): # loop through all the th elements
    
    item = (item.text).rstrip("\n")
    headings.append(item)
    
print(headings)

['Postal Code', 'Borough', 'Neighbourhood']


In [8]:

#with headings set work on the rest of the rows

every_row = [] # list for all rows
for row_num in range(len(body_rows)):
    row = [] 
    for row_item in body_rows[row_num].find_all("td"): 
        aa = re.sub("(\xa0)|(\n)|,","",row_item.text)
        row.append(aa)
    every_row.append(row)

In [9]:

df = pd.DataFrame(data=every_row,columns=headings)
pd.set_option('display.max_rows',180)
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park Harbourfront


In [10]:
df.replace('Not assigned',np.NaN, inplace=True)   # replace Not assigned with NaN

df.dropna(subset=['Borough'],inplace=True)  # drop all not assigned rows
df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park Harbourfront
5,M6A,North York,Lawrence Manor Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue Humber Valley Village
9,M1B,Scarborough,Malvern Rouge
11,M3B,North York,Don Mills
12,M4B,East York,Parkview Hill Woodbine Gardens
13,M5B,Downtown Toronto,Garden District Ryerson


In [11]:
df.shape

(103, 3)

In [12]:
#get the Geocoordinates
Geo_url='https://cocl.us/Geospatial_data'
geodf = pd.read_csv(Geo_url)
geodf

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [13]:
Geo_data = pd.merge(df,geodf, on=['Postal Code'])
Geo_data

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue Humber Valley Village,43.667856,-79.532242
6,M1B,Scarborough,Malvern Rouge,43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,Parkview Hill Woodbine Gardens,43.706397,-79.309937
9,M5B,Downtown Toronto,Garden District Ryerson,43.657162,-79.378937


In [46]:
toronto_data=Geo_data[Geo_data['Borough'].str.contains("Toronto")]
toronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,Regent Park Harbourfront,43.654260,-79.360636
4,M7A,Downtown Toronto,Queen's Park Ontario Provincial Government,43.662301,-79.389494
9,M5B,Downtown Toronto,Garden District Ryerson,43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [47]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="TO_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [48]:
# create map of Manhattan using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Geo_data['Latitude'], Geo_data['Longitude'], Geo_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [49]:
CLIENT_ID = 'GYE35LWRDA535NWYTHAIW5R05ZMK4SH2IRCERZTOXMUN2MT1' # your Foursquare ID
CLIENT_SECRET = '3IQEHJJ4EYZGWD5KIAT3NHOLNTKTMVWA4T5SZSOKHZGH0TKY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GYE35LWRDA535NWYTHAIW5R05ZMK4SH2IRCERZTOXMUN2MT1
CLIENT_SECRET:3IQEHJJ4EYZGWD5KIAT3NHOLNTKTMVWA4T5SZSOKHZGH0TKY


In [50]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [51]:

toronto_venues = getNearbyVenues(names=Geo_data['Neighbourhood'], latitudes=Geo_data['Latitude'], longitudes=Geo_data['Longitude'])


Parkwoods
Victoria Village
Regent Park Harbourfront
Lawrence Manor Lawrence Heights
Queen's Park Ontario Provincial Government
Islington Avenue Humber Valley Village
Malvern Rouge
Don Mills
Parkview Hill Woodbine Gardens
Garden District Ryerson
Glencairn
West Deane Park Princess Gardens Martin Grove Islington Cloverdale
Rouge Hill Port Union Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate Bloordale Gardens Old Burnhamthorpe Markland Wood
Guildwood Morningside West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor Wilson Heights Downsview North
Thorncliffe Park
Richmond Adelaide King
Dufferin Dovercourt Village
Scarborough Village
Fairview Henry Farm Oriole
Northwood Park York University
East Toronto Broadview North (Old East York)
Harbourfront East Union Station Toronto Islands
Little Portugal Trinity
Kennedy Park Ionview East Birchmount Park
Bayview Village
Downsvi

In [52]:
print(toronto_venues.shape)
toronto_venues.head()

(2112, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [53]:
#number of venues for each neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
Alderwood Long Branch,8,8,8,8,8,8
Bathurst Manor Wilson Heights Downsview North,24,24,24,24,24,24
Bayview Village,4,4,4,4,4,4
Bedford Park Lawrence Manor East,24,24,24,24,24,24
Berczy Park,56,56,56,56,56,56
Birch Cliff Cliffside West,4,4,4,4,4,4
Brockton Parkdale Village Exhibition Place,22,22,22,22,22,22
Business reply mail Processing Centre South Central Letter Processing Plant Toronto,15,15,15,15,15,15


In [54]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
toronto_onehot.shape

(2112, 265)

In [56]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,Agincourt,0.000000,0.000000,0.000000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
1,Alderwood Long Branch,0.000000,0.000000,0.000000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,Bathurst Manor Wilson Heights Downsview North,0.000000,0.000000,0.000000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
3,Bayview Village,0.000000,0.000000,0.000000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,Bedford Park Lawrence Manor East,0.000000,0.000000,0.000000,0.0000,0.000,0.0000,0.000,0.041667,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
5,Berczy Park,0.000000,0.000000,0.000000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Birch Cliff Cliffside West,0.000000,0.000000,0.000000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
7,Brockton Parkdale Village Exhibition Place,0.000000,0.000000,0.000000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
8,Business reply mail Processing Centre South Ce...,0.066667,0.000000,0.000000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,CN Tower King and Spadina Railway Lands Harbou...,0.000000,0.000000,0.062500,0.0625,0.125,0.1875,0.125,0.000000,0.000000,...,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00


In [57]:
num_top_venues = 15

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                              venue  freq
0         Latin American Restaurant  0.25
1                            Lounge  0.25
2                    Breakfast Spot  0.25
3                      Skating Rink  0.25
4         Middle Eastern Restaurant  0.00
5               Monument / Landmark  0.00
6   Molecular Gastronomy Restaurant  0.00
7        Modern European Restaurant  0.00
8                 Mobile Phone Shop  0.00
9                Miscellaneous Shop  0.00
10                      Yoga Studio  0.00
11               Mexican Restaurant  0.00
12                   Massage Studio  0.00
13                    Metro Station  0.00
14                      Men's Store  0.00


----Alderwood Long Branch----
                         venue  freq
0                  Pizza Place  0.25
1                     Pharmacy  0.12
2                 Dance Studio  0.12
3               Sandwich Place  0.12
4                          Pub  0.12
5                  Coffee Shop  0.12
6                  



----Glencairn----
                              venue  freq
0                       Pizza Place  0.17
1               Japanese Restaurant  0.17
2                  Asian Restaurant  0.17
3                               Pub  0.17
4                            Bakery  0.17
5                              Park  0.17
6                       Men's Store  0.00
7                     Metro Station  0.00
8                Mexican Restaurant  0.00
9         Middle Eastern Restaurant  0.00
10         Mediterranean Restaurant  0.00
11                Mobile Phone Shop  0.00
12       Modern European Restaurant  0.00
13                   Medical Center  0.00
14  Molecular Gastronomy Restaurant  0.00


----Golden Mile Clairlea Oakridge----
                              venue  freq
0                          Bus Line  0.22
1                            Bakery  0.22
2                      Soccer Field  0.11
3                              Park  0.11
4                      Intersection  0.11
5               

                  venue  freq
0           Coffee Shop  0.17
1                  Park  0.07
2                Bakery  0.07
3                   Pub  0.05
4        Breakfast Spot  0.05
5               Theater  0.05
6                  Café  0.05
7          Dessert Shop  0.02
8            Shoe Store  0.02
9            Beer Store  0.02
10                  Spa  0.02
11    French Restaurant  0.02
12       Chocolate Shop  0.02
13  Distribution Center  0.02
14       Farmers Market  0.02


----Richmond Adelaide King----
               venue  freq
0        Coffee Shop  0.12
1               Café  0.05
2         Restaurant  0.04
3      Deli / Bodega  0.03
4                Gym  0.03
5             Bakery  0.03
6    Thai Restaurant  0.03
7     Clothing Store  0.03
8   Asian Restaurant  0.02
9          Bookstore  0.02
10  Sushi Restaurant  0.02
11    Cosmetics Shop  0.02
12       Pizza Place  0.02
13     Burrito Place  0.02
14        Steakhouse  0.02


----Rosedale----
                              venue 

                              venue  freq
0                              Park   1.0
1                       Yoga Studio   0.0
2                Mexican Restaurant   0.0
3               Monument / Landmark   0.0
4   Molecular Gastronomy Restaurant   0.0
5        Modern European Restaurant   0.0
6                 Mobile Phone Shop   0.0
7                Miscellaneous Shop   0.0
8         Middle Eastern Restaurant   0.0
9                     Metro Station   0.0
10                            Motel   0.0
11                      Men's Store   0.0
12         Mediterranean Restaurant   0.0
13                   Medical Center   0.0
14                   Massage Studio   0.0


----York Mills West----
                              venue  freq
0                 Electronics Store  0.33
1                              Park  0.33
2                 Convenience Store  0.33
3                       Yoga Studio  0.00
4   Molecular Gastronomy Restaurant  0.00
5        Modern European Restaurant  0.00
6       

In [58]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [59]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Lounge,Breakfast Spot,Skating Rink,Middle Eastern Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop
1,Alderwood Long Branch,Pizza Place,Pharmacy,Dance Studio,Sandwich Place,Pub,Coffee Shop,Gym,Metro Station,Mexican Restaurant,Middle Eastern Restaurant
2,Bathurst Manor Wilson Heights Downsview North,Bank,Coffee Shop,Middle Eastern Restaurant,Sushi Restaurant,Supermarket,Frozen Yogurt Shop,Fried Chicken Joint,Bridal Shop,Shopping Mall,Pharmacy
3,Bayview Village,Japanese Restaurant,Café,Bank,Chinese Restaurant,Museum,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant
4,Bedford Park Lawrence Manor East,Sandwich Place,Coffee Shop,Italian Restaurant,Hobby Shop,Indian Restaurant,Butcher,Café,Liquor Store,Restaurant,Sushi Restaurant


**Cluster Nieghborhoods**

In [60]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [74]:
# add clustering labels
neighborhoods_venues_sorted.drop(['Cluster Labels'], axis=1, inplace=True)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighborhood')

toronto_merged.head() 

KeyError: "None of ['Neighbourhood'] are in the columns"

In [65]:
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Agincourt,Latin American Restaurant,Lounge,Breakfast Spot,Skating Rink,Middle Eastern Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop
1,1,Alderwood Long Branch,Pizza Place,Pharmacy,Dance Studio,Sandwich Place,Pub,Coffee Shop,Gym,Metro Station,Mexican Restaurant,Middle Eastern Restaurant
2,1,Bathurst Manor Wilson Heights Downsview North,Bank,Coffee Shop,Middle Eastern Restaurant,Sushi Restaurant,Supermarket,Frozen Yogurt Shop,Fried Chicken Joint,Bridal Shop,Shopping Mall,Pharmacy
3,1,Bayview Village,Japanese Restaurant,Café,Bank,Chinese Restaurant,Museum,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant
4,1,Bedford Park Lawrence Manor East,Sandwich Place,Coffee Shop,Italian Restaurant,Hobby Shop,Indian Restaurant,Butcher,Café,Liquor Store,Restaurant,Sushi Restaurant


In [72]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

KeyError: 'Cluster Labels'